In [9]:
import pandas as pd
import nltk
import numpy as np

# Get Data

In [11]:
review_reader = pd.read_json('yelp_academic_dataset_review.json', chunksize=1, lines=True)
i = 0
business_data = pd.DataFrame([])
for chunk in review_reader:
    #print(chunk)
    business_data = chunk
    i = i+1
    if i==1:
        break
#business_data = pd.concat(df)
#print(type(chunk))
business_data

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g


In [12]:
business_id = business_data['business_id']
business_id = business_id[0]
business_id

'iCQpiavjjPzJ5_3gPD5Ebg'

In [ ]:
'''review_reader = pd.read_json('yelp_academic_dataset_review.json', chunksize=1, lines=True)
i = 0
j = 0
df = []
for chunk in review_reader:
    #for c in chunk:
    if chunk['business_id'][j] == business_id:
        #print(chunk['business_id'][j])
        df.append(chunk)
        i = i+1
        if i==200:
            break
    j = j+1
    #if j==10000:
    #    break'''

In [28]:
#review_data = pd.concat(df)
review_data = pd.read_csv('review_data.csv')
review_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 1000571
Data columns (total 9 columns):
business_id    395 non-null object
cool           395 non-null int64
date           395 non-null datetime64[ns]
funny          395 non-null int64
review_id      395 non-null object
stars          395 non-null int64
text           395 non-null object
useful         395 non-null int64
user_id        395 non-null object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 30.9+ KB


## Write data to csv file

In [31]:
#review_data.to_csv('review_data.csv')
#review_text.to_json('review_text.json')

## Cleaning data

In [34]:
reviews = review_data.loc[:,['review_id','text']]
reviews.head()

,review_id,text
0,x7mDIiDB3jEiPGPHOmDzyw,The pizza was okay. Not the best I've had. I p...
788,VluIpojME0yKOcRKI5L0Iw,"came here on Monday, no line. its good, but no..."
3314,nsB9JAeghk0zOaSulSm9Yw,This place is truly a secret! Its so hidden t...
6726,pcn01EMERpCpHEcoaohdEg,This SECRET PIZZA was a secret for about 5 st...
8969,0mFtAPTmInbXHqDjX9eiOg,this is the BEST place to grab a quick slice o...


In [69]:
txt = []
for index,row in reviews.iterrows():
    text = row['text']
    text = text.split('. ')
    txt.extend(text)
txt
review_text = pd.DataFrame(txt, columns=['text'])
review_text.head()

,text
0,The pizza was okay
1,Not the best I've had
2,I prefer Biaggio's on Flamingo / Fort Apache
3,The chef there can make a MUCH better NY style...
4,The pizzeria @ Cosmo was over priced for the q...


# Tokenize

In [70]:
#nltk.download('punkt')
token_list = []
for index,row in review_text.iterrows():
    text = row['text']
    tokens = nltk.word_tokenize(text)
    token_list.append([tokens])
    #print(token_list)
token_col = pd.DataFrame(token_list, columns=['tokens'])
#print(token_list)
#token_col.head()
review_text = pd.concat([review_text, token_col], axis=1)
review_text.head()

,text,tokens
0,The pizza was okay,"[The, pizza, was, okay]"
1,Not the best I've had,"[Not, the, best, I, 've, had]"
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort..."
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,..."
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f..."


# 3.1 POS Tagging

## Stopword removal and conversion to lowercase
### (not needed)

In [71]:
'''#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
ftext = []
for index,row in review_text.iterrows():
    tokens = row['tokens']
    filtered_text = [w.lower() for w in tokens]#if not w in stop_words] 
    ftext.append([filtered_text])
filtered_col = pd.DataFrame(ftext, columns=['filtered_tokens'])
review_text = pd.concat([review_text, filtered_col], axis=1)
review_text.head()'''

,text,tokens,filtered_tokens
0,The pizza was okay,"[The, pizza, was, okay]","[the, pizza, was, okay]"
1,Not the best I've had,"[Not, the, best, I, 've, had]","[not, the, best, i, 've, had]"
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort...","[i, prefer, biaggio, 's, on, flamingo, /, fort..."
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,...","[the, chef, there, can, make, a, much, better,..."
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f...","[the, pizzeria, @, cosmo, was, over, priced, f..."


## Tagging using upenn tagset

In [72]:
#nltk.download('averaged_perceptron_tagger')
ttext = []
for index,row in review_text.iterrows():
    filtered_text = row['filtered_tokens']
    tagged_text = nltk.pos_tag(filtered_text) 
    ttext.append([tagged_text])
tagged_col = pd.DataFrame(ttext, columns=['tagged_tokens'])
review_text = pd.concat([review_text, tagged_col], axis=1)
review_text.head()

,text,tokens,filtered_tokens,tagged_tokens
0,The pizza was okay,"[The, pizza, was, okay]","[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]"
1,Not the best I've had,"[Not, the, best, I, 've, had]","[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (..."
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort...","[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P..."
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,...","[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)..."
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f...","[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ..."


In [9]:
#nltk.download('tagsets')
#nltk.help.upenn_tagset('NN')

NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...


## Noun / noun phrase filtering

In [73]:
ntext = []
for index,row in review_text.iterrows():
    tagged_text = row['tagged_tokens']
    nouns = []
    for tt in tagged_text:
        tag = tt[1]
        if tag in ['NN','NNS','NNP','NNPS']:
            nouns.append(tt[0])
    ntext.append([nouns])
noun_col = pd.DataFrame(ntext, columns=['nouns'])
noun_col
review_text = pd.concat([review_text, noun_col], axis=1)
review_text.head()

,text,tokens,filtered_tokens,tagged_tokens,nouns
0,The pizza was okay,"[The, pizza, was, okay]","[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]",[pizza]
1,Not the best I've had,"[Not, the, best, I, 've, had]","[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (...",[i]
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort...","[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P...","[i, biaggio, flamingo, /, fort, apache]"
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,...","[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)...","[chef, style, pizza]"
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f...","[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ...","[pizzeria, @, cosmo, quality, lack, personalit..."


## Stemming 
### (not including in procedure)

In [11]:
'''from nltk.stem import PorterStemmer
ps = PorterStemmer()
for w in nouns:
    print(w + '\t:\t' + ps.stem(w))'''

pizza	:	pizza
okay	:	okay
Biaggio	:	biaggio
Flamingo	:	flamingo
/	:	/
Fort	:	fort
Apache	:	apach
chef	:	chef
MUCH	:	much
NY	:	NY
style	:	style
pizza	:	pizza
pizzeria	:	pizzeria
@	:	@
Cosmo	:	cosmo
quality	:	qualiti
lack	:	lack
personality	:	person
food	:	food
Biaggio	:	biaggio
youre	:	your
family	:	famili
home	:	home
recipes	:	recip
people	:	peopl
CARE	:	care
food	:	food
casino	:	casino


## Fuzzy matching (example)
### (not including in procedure)

In [12]:
'''from fuzzywuzzy import fuzz
from fuzzywuzzy import process
query = 'Barack Obama'
choices = ['Barack H Obama', 'Barack H. Obama', 'B. Obama']
# Get a list of matches ordered by score, default limit to 5
process.extract(query, choices)
# [('Barack H Obama', 95), ('Barack H. Obama', 95), ('B. Obama', 85)]
 
# If we want only the top one
process.extractOne(query, choices)
# ('Barack H Obama', 95)'''

('Barack H Obama', 95)

## Writing processed data to CSV

In [40]:
review_text.to_csv('review_text.csv')

## Running additional noun filtering
### (not needed)

In [74]:
'''ttext = []
for index,row in review_text.iterrows():
    filtered_text = row['nouns']
    tagged_text = nltk.pos_tag(filtered_text) 
    ttext.append([tagged_text])
tagged_col = pd.DataFrame(ttext, columns=['tagged_noun_tokens'])
review_text = pd.concat([review_text, tagged_col], axis=1)
review_text.head()'''

,text,tokens,filtered_tokens,tagged_tokens,nouns,tagged_noun_tokens
0,The pizza was okay,"[The, pizza, was, okay]","[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]",[pizza],"[(pizza, NN)]"
1,Not the best I've had,"[Not, the, best, I, 've, had]","[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (...",[i],"[(i, NN)]"
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort...","[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P...","[i, biaggio, flamingo, /, fort, apache]","[(i, NN), (biaggio, VBP), (flamingo, NN), (/, ..."
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,...","[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)...","[chef, style, pizza]","[(chef, NN), (style, NN), (pizza, NN)]"
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f...","[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ...","[pizzeria, @, cosmo, quality, lack, personalit...","[(pizzeria, NNS), (@, VBP), (cosmo, JJ), (qual..."


In [75]:
'''ntext = []
for index,row in review_text.iterrows():
    tagged_text = row['tagged_noun_tokens']
    nouns = []
    for tt in tagged_text:
        tag = tt[1]
        if tag in ['NN','NNS','NNP','NNPS']:
            nouns.append(tt[0])
    ntext.append([nouns])
noun_col = pd.DataFrame(ntext, columns=['filtered_nouns'])
noun_col
review_text = pd.concat([review_text, noun_col], axis=1)
review_text.head()'''

,text,tokens,filtered_tokens,tagged_tokens,nouns,tagged_noun_tokens,filtered_nouns
0,The pizza was okay,"[The, pizza, was, okay]","[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]",[pizza],"[(pizza, NN)]",[pizza]
1,Not the best I've had,"[Not, the, best, I, 've, had]","[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (...",[i],"[(i, NN)]",[i]
2,I prefer Biaggio's on Flamingo / Fort Apache,"[I, prefer, Biaggio, 's, on, Flamingo, /, Fort...","[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P...","[i, biaggio, flamingo, /, fort, apache]","[(i, NN), (biaggio, VBP), (flamingo, NN), (/, ...","[i, flamingo, /, fort, apache]"
3,The chef there can make a MUCH better NY style...,"[The, chef, there, can, make, a, MUCH, better,...","[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)...","[chef, style, pizza]","[(chef, NN), (style, NN), (pizza, NN)]","[chef, style, pizza]"
4,The pizzeria @ Cosmo was over priced for the q...,"[The, pizzeria, @, Cosmo, was, over, priced, f...","[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ...","[pizzeria, @, cosmo, quality, lack, personalit...","[(pizzeria, NNS), (@, VBP), (cosmo, JJ), (qual...","[pizzeria, quality, lack, personality, food]"


# 3.2 Frequent features identification

In [76]:
filtered_nouns = review_text['nouns']
filtered_nouns = filtered_nouns.tolist()
filtered_nouns[0:5]

[['pizza'],
 ['i'],
 ['i', 'biaggio', 'flamingo', '/', 'fort', 'apache'],
 ['chef', 'style', 'pizza'],
 ['pizzeria', '@', 'cosmo', 'quality', 'lack', 'personality', 'food']]

In [77]:
features = {}
print(features.keys())
for nouns in filtered_nouns:
    keys = features.keys()
    visited = []
    for n in nouns:
        if n not in visited:
            if n in keys:
                features[n] += 1
            else:
                features.update({n: 1})

#features

dict_keys([])


In [62]:
frequent_ftr = []
print(review_text['text'].count())
threshold = int(review_text['text'].count() * 0.015)
for k,v in features.items():
    if v >= threshold:
        frequent_ftr.append(k)
frequent_ftr

1867


['pizza',
 'floor',
 'i',
 'service',
 'line',
 'slices',
 'slice',
 'staff',
 'style',
 'people',
 'crust',
 'time',
 'pie',
 'place',
 'order',
 'strip',
 'spot',
 'hallway',
 'cosmo',
 'wait',
 'pepperoni',
 'way',
 'sausage',
 'food',
 'eat',
 'york',
 'vegas',
 'night',
 'price']